In [1]:
# Will use xgboot model for analysis, no need to transform data.

In [2]:
import numpy as np
import pandas as pd
from scipy import stats
from util import plot_helper, iv_helper, ft_helper
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
data_processed = pd.read_csv('data/loan_sample_processed_1pct.csv', index_col=0)
data_processed.head()

,collection_recovery_fee,recoveries,last_fico_range_low,last_fico_range_high,last_pymnt_amnt,total_rec_prncp,total_pymnt_inv,total_pymnt,int_rate,total_rec_late_fee,...,installment,mort_acc,debt_settlement_flag,last_credit_pull_d,sub_grade,grade,last_pymnt_d,term,home_ownership,loan_status_code
1,0.0,0.0,680.0,684.0,6294.65,8700.0,9805.21,9861.890000,13.98,0.0,...,297.27,0.0,N,Jan-2019,C3,C,Jun-2015,36 months,RENT,0
2,0.0,0.0,685.0,689.0,476.23,15000.0,17153.81,17153.806653,8.90,0.0,...,476.30,3.0,N,Apr-2017,A5,A,Apr-2017,36 months,OWN,0
3,0.0,0.0,570.0,574.0,9561.67,12000.0,13251.85,13251.845181,13.98,0.0,...,410.02,0.0,N,May-2018,C1,C,Oct-2014,36 months,RENT,0
5,0.0,0.0,700.0,704.0,623.93,20000.0,23891.06,23891.056999,12.39,0.0,...,668.02,0.0,N,Jan-2019,C1,C,Dec-2017,36 months,RENT,0
6,0.0,0.0,750.0,754.0,334.49,10000.0,11977.77,11977.770391,12.12,0.0,...,332.72,2.0,N,Jan-2017,B3,B,Oct-2015,36 months,MORTGAGE,0


In [4]:
data_processed.describe()

,collection_recovery_fee,recoveries,last_fico_range_low,last_fico_range_high,last_pymnt_amnt,total_rec_prncp,total_pymnt_inv,total_pymnt,int_rate,total_rec_late_fee,...,tot_hi_cred_lim,loan_amnt,funded_amnt,funded_amnt_inv,avg_cur_bal,tot_cur_bal,bc_util,installment,mort_acc,loan_status_code
count,9297.000000,9297.000000,9297.000000,9297.000000,9297.000000,9297.000000,9297.000000,9297.000000,9297.000000,9297.000000,...,9.297000e+03,9297.000000,9297.000000,9297.000000,9297.000000,9.297000e+03,9297.000000,9297.000000,9297.000000,9297.000000
mean,45.933611,273.755918,658.817360,675.489405,5861.947528,12382.984043,15125.090475,15131.078062,13.448199,1.642699,...,1.832550e+05,14710.078520,14710.078520,14704.213725,13734.935033,1.489887e+05,59.518780,445.285260,1.764655,0.200065
std,193.026194,1094.433299,130.766237,80.708628,7250.186264,8922.180932,10439.674140,10443.125774,4.801350,10.654913,...,1.717149e+05,8728.950581,8728.950581,8725.656121,14945.792123,1.535220e+05,28.462775,261.857754,2.027433,0.400070
min,0.000000,0.000000,0.000000,499.000000,0.000000,0.000000,151.130000,151.130000,5.310000,0.000000,...,2.000000e+02,1000.000000,1000.000000,1000.000000,0.000000,0.000000e+00,0.000000,30.870000,0.000000,0.000000
25%,0.000000,0.000000,620.000000,624.000000,456.260000,5200.000000,7060.460000,7060.800000,9.990000,0.000000,...,5.512800e+04,8000.000000,8000.000000,8000.000000,3365.000000,3.396800e+04,37.900000,257.040000,0.000000,0.000000
50%,0.000000,0.000000,690.000000,694.000000,2799.450000,10000.000000,12457.770000,12458.850790,12.990000,0.000000,...,1.264230e+05,12575.000000,12575.000000,12525.000000,8284.000000,9.444400e+04,62.700000,382.630000,1.000000,0.000000
75%,0.000000,0.000000,730.000000,734.000000,9121.100000,18000.000000,20863.070000,20872.412415,16.290000,0.000000,...,2.644800e+05,20000.000000,20000.000000,20000.000000,19377.000000,2.210090e+05,84.300000,583.550000,3.000000,0.000000
max,5638.091400,31322.730000,845.000000,850.000000,40622.850000,40000.000000,60580.440000,60580.436136,30.990000,298.990001,...,1.581836e+06,40000.000000,40000.000000,40000.000000,160233.000000,1.447807e+06,161.400000,1503.890000,25.000000,1.000000


In [7]:
numberical_ft=list(data_processed.dtypes.loc[lambda x : x == 'float64'].to_dict().keys())
# len(numberical_ft)
# numberical_ft

## Transform numberical data

In [8]:
all_features = numberical_ft + ['loan_status_code']

In [9]:
data_processed_numberical = data_processed[all_features]

In [10]:
data_processed_numberical.head()

,collection_recovery_fee,recoveries,last_fico_range_low,last_fico_range_high,last_pymnt_amnt,total_rec_prncp,total_pymnt_inv,total_pymnt,int_rate,total_rec_late_fee,...,tot_hi_cred_lim,loan_amnt,funded_amnt,funded_amnt_inv,avg_cur_bal,tot_cur_bal,bc_util,installment,mort_acc,loan_status_code
1,0.0,0.0,680.0,684.0,6294.65,8700.0,9805.21,9861.890000,13.98,0.0,...,66984.0,8700.0,8700.0,8650.0,4955.0,59465.0,76.8,297.27,0.0,0
2,0.0,0.0,685.0,689.0,476.23,15000.0,17153.81,17153.806653,8.90,0.0,...,20300.0,15000.0,15000.0,15000.0,2094.0,12562.0,74.3,476.30,3.0,0
3,0.0,0.0,570.0,574.0,9561.67,12000.0,13251.85,13251.845181,13.98,0.0,...,28701.0,12000.0,12000.0,12000.0,2925.0,17547.0,111.0,410.02,0.0,0
5,0.0,0.0,700.0,704.0,623.93,20000.0,23891.06,23891.056999,12.39,0.0,...,88645.0,20000.0,20000.0,20000.0,8759.0,70075.0,85.1,668.02,0.0,0
6,0.0,0.0,750.0,754.0,334.49,10000.0,11977.77,11977.770391,12.12,0.0,...,199608.0,10000.0,10000.0,10000.0,20383.0,183444.0,90.5,332.72,2.0,0


In [11]:
data_processed_numberical.dtypes

collection_recovery_fee    float64
recoveries                 float64
last_fico_range_low        float64
last_fico_range_high       float64
last_pymnt_amnt            float64
total_rec_prncp            float64
total_pymnt_inv            float64
total_pymnt                float64
int_rate                   float64
total_rec_late_fee         float64
fico_range_low             float64
fico_range_high            float64
total_rec_int              float64
bc_open_to_buy             float64
total_rev_hi_lim           float64
dti                        float64
total_bc_limit             float64
tot_hi_cred_lim            float64
loan_amnt                  float64
funded_amnt                float64
funded_amnt_inv            float64
avg_cur_bal                float64
tot_cur_bal                float64
bc_util                    float64
installment                float64
mort_acc                   float64
loan_status_code             int64
dtype: object

In [12]:
for feature_name in numberical_ft:
    if feature_name != 'loan_status_code':
        xs = data_processed_numberical[feature_name]
        zs_linear = ft_helper.transform_linear(xs)
        zs_norm = ft_helper.transform_norm(xs)
        zs_boxcox = ft_helper.transform_boxcox(xs)
    
        data_processed_numberical[f'{feature_name}_linear'] = zs_linear
        data_processed_numberical[f'{feature_name}_norm'] = zs_norm
        data_processed_numberical[f'{feature_name}_boxcox'] = zs_boxcox

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/

In [13]:
data_processed_numberical.to_csv('data/data_transformed.csv')

In [14]:
data_processed_numberical.head()

,collection_recovery_fee,recoveries,last_fico_range_low,last_fico_range_high,last_pymnt_amnt,total_rec_prncp,total_pymnt_inv,total_pymnt,int_rate,total_rec_late_fee,...,tot_cur_bal_boxcox,bc_util_linear,bc_util_norm,bc_util_boxcox,installment_linear,installment_norm,installment_boxcox,mort_acc_linear,mort_acc_norm,mort_acc_boxcox
1,0.0,0.0,680.0,684.0,6294.65,8700.0,9805.21,9861.890000,13.98,0.0,...,1.983789,0.475836,0.607184,73.837560,0.180853,-0.565281,14.955028,0.00,-0.870436,0.000000
2,0.0,0.0,685.0,689.0,476.23,15000.0,17153.81,17153.806653,8.90,0.0,...,0.821638,0.460347,0.519346,71.459707,0.302392,0.118448,17.705560,0.12,0.609347,1.178778
3,0.0,0.0,570.0,574.0,9561.67,12000.0,13251.85,13251.845181,13.98,0.0,...,1.025667,0.687732,1.808818,106.288027,0.257396,-0.134681,16.788838,0.00,-0.870436,0.000000
5,0.0,0.0,700.0,704.0,623.93,20000.0,23891.06,23891.056999,12.39,0.0,...,2.132915,0.527261,0.898809,81.725855,0.432547,0.850640,19.932492,0.00,-0.870436,0.000000
6,0.0,0.0,750.0,754.0,334.49,10000.0,11977.77,11977.770391,12.12,0.0,...,3.069689,0.560719,1.088541,86.853228,0.204919,-0.429895,15.577496,0.08,0.116086,0.965394
